# Economicos

In [1]:
%%capture
import sys
!ln -s ../syntheticml .
!{sys.executable} -m pip install -U -r ../requirements.txt
!{sys.executable} -m pip install -U kaleido

In [2]:
import pandas as pd
import numpy as np
df = pd.read_parquet('../datasets/economicos/raw/full_dedup_economicos_step0.parquet');
df.sample(3)

,url,description,price,property_type,transaction_type,state,county,publication_date,rooms,bathrooms,m_built,m_size,source,title,address,owner,_price
613115,https://www.economicos.cl/propiedades/se-vende...,Se vende terreno de 3000 mt2 en la comuna del ...,$ 45.000.000,Sitio o Terreno,Venta,Valparaíso,El Tabo,2019-12-06 11:55:30,NaN,NaN,NaN,3000.0,None,"Se vende terreno, en el Tabo",Se vende terreno de 3000 mt2 en la comuna del ...,Daniela Muñoz,1591.94322
130347,https://www.economicos.cl/propiedades/arriendo...,Arriendo Departamento Amoblado Full a pasos de...,$ 400.000,Departamento Amoblado,Arriendo,Tarapacá,Iquique,2019-11-06 13:30:38,1.0,1.0,NaN,NaN,None,None,None,None,14.25245
529874,https://www.economicos.cl/propiedades/local-o-...,Oficinas Workspaces Solutions Fernandez Luco (...,89 UF,Local o Casa comercial,Arriendo,Metropolitana de Santiago,Vitacura,2022-02-23 00:00:08,NaN,2.0,158.0,NaN,El Mercurio,Local o Casa comercial en Arriendo en Vitacura...,"Kennedy Lateral // Luis Carrera Vitacura, Metr...",Nexxos,89.00000


In [3]:
%%capture
from syntheticml.data.synthetic import Synthetic, MODELS
category_columns=("property_type", "transaction_type", "state", "county", "rooms", "bathrooms", "m_built", "m_size", "source", )
# TODO: Estudiar implicancia de valores nulos en categorias y numeros
df_converted = df.fillna(dict(
        property_type = "None",
        transaction_type = "None",
        state = "None",
        county = "None",
        rooms = -1,
        bathrooms = -1,
        m_built = -1,
        m_size = -1,
        source = "None"
)).fillna(-1).astype({k: 'str' for k in ("description", "price", "title", "address", "owner",)})
print(df_converted.shape)
basedate = pd.Timestamp('2017-12-01')
dtime = df_converted.pop("publication_date")
df_converted["publication_date"] = dtime.apply(lambda x: (x - basedate).days)
syn = Synthetic(df_converted, 
        id="url", 
        category_columns=category_columns,
        text_columns=("description", "price", "title", "address", "owner", ),
        exclude_columns=tuple(),
        synthetic_folder = "../datasets/economicos/synthb",
        models=['copulagan', 'tvae', 'gaussiancopula', 'ctgan', 'smote-enc', 'tddpm_mlp'],
        n_sample = df_converted.shape[0],
        target_column="_price",
        max_cpu_pool=1
)


In [4]:
%%capture
syn.process()
#syn.process(remaining_columns=("property_type", "transaction_type", "state",))
syn.process_scores()


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

: 

## Metricas
### Continuas

In [ ]:
metrics = syn.current_metrics()
metrics[~metrics.is_categorical].dropna(axis=1, how='all')

### Categoricas

In [ ]:
metrics[metrics.is_categorical].dropna(axis=1, how='all')

## Scores

In [ ]:
#best_model = "smote-enc_noise_682338"
best_model = "tddpm_mlp_27574"
second_best= "smote-enc_27574"
avg = syn.scores[syn.scores["type"] == "avg"]
avg.sort_values("score", ascending=False).loc[["tddpm_mlp_27574","smote-enc_27574","copulagan_27574","ctgan_27574","gaussiancopula_noise_27574","tvae_noise_27574","tvae_27574","ctgan_noise_27574","copulagan_noise_27574","gaussiancopula_27574"],["score", "DCR ST 5th", "DCR SH 5th", "DCR TH 5th"]]

In [ ]:
from syntheticml.data.charts import Charts
for fig in syn.get_charts(best_model, {'date', 'id', 'zipcode', 'lat', 'long', 'yr_renovated'}):
    if fig:
        display(fig.show("png"))

In [ ]:
syn.charts.pair_corr(df.loc[:, syn.fake_data[best_model].columns].dropna(), syn.fake_data[best_model].astype({k:np.float64 for k in ('bathrooms','rooms','m_built','m_size')}), {'id', 'waterfront', 'yr_renovated'}, "_price").update_layout(dict(width=1000)).show("png")

In [ ]:
syn.fake_data[best_model]

In [ ]:
syn.fake_data[second_best]

In [ ]:
from syntheticml.data.charts import Charts
for fig in syn.get_charts(second_best, {'date', 'id', 'zipcode', 'lat', 'long', 'yr_renovated'}):
    if fig:
        display(fig.show("png"))